#### Setup

In [ ]:
import pandas as pd
import numpy as np

import os
import glob

from PIL import Image

In [ ]:
path = 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp/'
directory = os.listdir(path)

#### Image resizing and moving

In [ ]:
# resize all images in the folder to standard size of 500 x 500
# new images have '_renamed' at the end of the og filename

def resize_img():
    for item in directory:
        if os.path.isfile(path+item):
            im = Image.open(path+item)
            f, e = os.path.splitext(path+item)
            imResize = im.resize((500, 500), Image.ANTIALIAS) #this smooths the image edges, maybe dont do? not sure
            imResize.save(f + '_resized.png', 'PNG', quality = 90) #95 is recommended max, 75 is avg?

In [ ]:
#### ONLY RUN THIS WHEN YOU NEED TO RANDOMIZE A NEW FOLDER
resize_img()

In [ ]:
# move the resized images to a new folder
# NSLT_CT_IMG_samp_resize
import shutil

for file in glob.glob('C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp/*resized.png'):
    shutil.move(file, 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize/')

In [ ]:
# new directory of resized images --> /NSLT_CT_IMG_samp_resize/

os.getcwd()

In [ ]:
# new path
path = 'C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize/'

In [ ]:
# grab filenames and put into a list

def get_imlist(path):
    return [os.path.join(path,f) for f in os.listdir(path) if f.endswith('.png')]

In [ ]:
filepaths = get_imlist(path)
filepaths

In [ ]:
filepaths_2 = [item.split('/') for item in filepaths]
filepaths_2

In [ ]:
#keep only the filename
filepaths_3 = [item.pop() for item in filepaths_2]
filepaths_3

# rename
filenames_imgs = filepaths_3

#### Converting resized images into arrays

In [ ]:
images = []

# make empty array --> 500x500 size
np_imgs = np.zeros((500, 500))

In [ ]:
# convert each image into an array
count = 0

for f in glob.iglob('C:/Users/aks13/Dropbox (Partners HealthCare)/ML stuff/NSLT_CT_IMG_samp_resize/*.png'):
    images = Image.open(f).convert('L') #convert to b/w THEN flatten, L is grayscale
    images = np.asarray(images) # create the arrays...
    images = [item.flatten() for item in images] #...then flatten before adding
    np_imgs[count,:] #flattened image vector
    count = count + 1
    
images = np.array(images)

# add back filenames to this and make a new list
# 1:1 ratio filename:array once flattened to 1D

In [ ]:
len(images)

In [ ]:
images

In [ ]:
len(filenames_imgs)
# these two lists have to match in length or they wont combine correctly

#### Convert arrays into a dataframe, matching filenames as the index

In [ ]:
# try --> just have the df from the flat_imgs arrays first
# then insert the filenames as the first column

df_v1 = pd.DataFrame(images, index=filenames_imgs)
#df_v1 = pd.DataFrame(images[0:10], index=filenames_imgs[0:10]) #test 10 output

df_v1

#### PCA

In [ ]:
from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler #dont scale outliers lol

X = df_v1.values
X.shape

In [ ]:
X

In [ ]:
# old scaled method
# trying without

# scale the data
##scaler = StandardScaler()
##scaler.fit(X)
##X_scaled = scaler.transform(X)
##X_scaled

In [ ]:
pca_v1 = PCA(n_components=5, random_state=42)
pca_v1.fit(X)
#pca_v1.fit(X_scaled)

In [ ]:
X_pca_v1 = pca_v1.transform(X)
#X_pca_v1 = pca_v1.transform(X_scaled)
X_pca_v1.shape

In [ ]:
X #original df from pca arrays
df_v1 # before doing anything
X_pca_v1 #array of PCA values that were fit/transformed --> use this in IF

In [ ]:
df_v1.info()

#### Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
# put the PCA array back to a DF to preserve the new components

pca_v1_df = pd.DataFrame(X_pca_v1, index=filenames_imgs)
pca_v1_df.columns = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5']
pca_v1_df

In [ ]:
anomaly_pca = ['PCA 1', 'PCA 2', 'PCA 3', 'PCA 4', 'PCA 5']

if_model = IsolationForest(max_samples=100, contamination=0.1, random_state=42)
if_model.fit(pca_v1_df[anomaly_pca]) # use the PCA'd df

In [ ]:
pca_v1_df['anomaly score'] = if_model.decision_function(pca_v1_df[anomaly_pca])
pca_v1_df['anomaly'] = if_model.predict(pca_v1_df[anomaly_pca])
pca_v1_df

In [ ]:
pca_v1_df # -1 = outlier (anomaly), 1 = non-outlier (normal)

In [ ]:
#pca_v1_df.to_csv('nonscaled-check3.csv')